In [1]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from argparse import Namespace
from HH4b.postprocessing.PostProcess import load_process_run3_samples

In [ ]:
import pandas as pd
import pyarrow.parquet as pq


def inspect_parquet_file(file_path):
    """Inspect the contents of a Parquet file."""
    try:
        print(f"\nInspecting Parquet file: {file_path}\n")

        # Open the Parquet file metadata
        parquet_file = pq.ParquetFile(file_path)

        # Display general file metadata
        print("File Metadata:")
        print(f"  Number of Rows: {parquet_file.metadata.num_rows}")
        print(f"  Number of Columns: {parquet_file.metadata.num_columns}")
        print(f"  Number of Row Groups: {parquet_file.num_row_groups}")

        # Display schema information
        print("\nSchema:")
        for col in range(parquet_file.schema):
            print(f"  - {col}")

        # Display column names
        print("\nColumns:")
        for column in parquet_file.schema.names:
            print(f"  - {column}")

        # Optional: Read a sample of the data (first 5 rows)
        print("\nPreview of the first 5 rows of data:")
        df = pd.read_parquet(file_path)
        print(df.head())

    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


inspect_parquet_file(
    "/ceph/cms/store/user/cmantill/bbbb/skimmer/24Sep25_v12v2_private_signal/2022/QCD_HT-400to600/parquet/out_1.parquet"
)


Inspecting Parquet file: /ceph/cms/store/user/cmantill/bbbb/skimmer/24Sep25_v12v2_private_signal/2022/QCD_HT-400to600/parquet/out_1.parquet

File Metadata:
  Number of Rows: 83
  Number of Columns: 256
  Number of Row Groups: 1

Schema:
An error occurred: 'pyarrow._parquet.ParquetSchema' object has no attribute 'num_columns'


In [ ]:
data_folder = "24May24_v12_private_signal"
args = Namespace(
    templates_tag="24June27",
    data_dir="/ceph/cms/store/user/cmantill/bbbb/skimmer/",
    # tag=data_folder,
    # years=["2022"],
    training_years=None,
    mass="H2PNetMass",
    bdt_model="24May31_lr_0p02_md_8_AK4Away",
    bdt_config="24May31_lr_0p02_md_8_AK4Away",
    txbb_wps=[0.975, 0.92],
    bdt_wps=[0.98, 0.88, 0.03],
    method="sideband",
    vbf_txbb_wp=0.95,
    vbf_bdt_wp=0.98,
    sig_keys=["hh4b", "vbfhh4b"],
    pt_first=300,
    pt_second=250,
    bdt_roc=False,
    control_plots=False,
    fom_scan=False,
    fom_scan_bin1=True,
    fom_scan_bin2=True,
    fom_scan_vbf=False,
    templates=False,
    legacy=True,
    vbf=True,
    vbf_priority=False,
    weight_ttbar_bdt=1,
    blind=True,
)

In [ ]:
bdt_training_keys = ["qcd", "vbfhh4b-k2v0", "hh4b", "ttbar"]
mass_window = np.array([105, 150])
years = ["2022", "2022EE", "2023", "2023BPix"]

ev_dicts = []
for year in years:
    ev_dict, _ = load_process_run3_samples(
        args,
        year=year,
        bdt_training_keys=bdt_training_keys,
        control_plots=False,
        plot_dir="plot_dir",
        mass_window=mass_window,
    )
    ev_dicts.append((year, ev_dict))

"""
python3 PostProcess.py --templates-tag 24June27 --tag 24May24_v12_private_signal --mass H2PNetMass --legacy --bdt-config 24May31_lr_0p02_md_8_AK4Away --bdt-model 24May31_lr_0p02_md_8_AK4Away --txbb-wps 0.975 0.92 --bdt-wps 0.98 0.88 0.03 --vbf-txbb-wp 0.95 --vbf-bdt-wp 0.98 --no-bdt-roc --no-fom-scan --no-fom-scan-bin2 --no-fom-scan-bin1 --data-dir /ceph/cms/store/user/cmantill/bbbb/skimmer/ --method abcd --no-vbf-priority --vbf --no-fom-scan-vbf --pt-second 250 --templates --years 2022 --sig-keys hh4b vbfhh4b
"""
# make array with event nr, signal category
# make mask with signal category cutoffs, apply to array

# save as root

In [ ]:
import uproot
import pandas as pd
import os

# select columns to extract for eventlist
eventlist_dict = [
    "event",
    "bdt_score",
    "bdt_score_vbf",
    "H2TXbb",
    "H2Msd",
    "run",
    "H2PNetMass",
    "luminosityBlock",
]
keys_to_save = ["hh4b", "vbfhh4b", "data"]

# Ensure the eventlist folder exists
eventlist_folder = "eventlist_files_2025Jan6"
os.makedirs(eventlist_folder, exist_ok=True)

# TODO: check how data is loaded (should reference the folder names somewhere)
# TODO: print eventlist_dict to see if you can get keys "data" and vbfhh4b
# Loop over all years and save event lists for each year in separate root files
for year, ev_dict in ev_dicts:
    for key in ev_dict.keys():
        if "data" in key or "hh4b" in key or "vbfhh4b" in key:
            print(key)
            tree_df = ev_dict[key]
            event_list = tree_df[eventlist_dict]
            array_to_save = {col: event_list[col].to_numpy() for col in event_list.columns}

            # Define the ROOT file path
            file_path = f"{eventlist_folder}/eventlist_boostedHH4b_{year}.root"

            # Check if the ROOT file already exists
            if os.path.exists(file_path):
                # File exists, use update mode to append the new tree
                with uproot.update(file_path) as file:
                    file[key] = array_to_save  # Append new tree
            else:
                # File doesn't exist, create a new one
                with uproot.recreate(file_path) as file:
                    file[key] = array_to_save  # Create the first tree

In [ ]:
# Double-check file contents
years = ["2022", "2022EE", "2023", "2023BPix"]
dfs_from_root = {}

for year in years:
    # Define the ROOT file path
    file_path = f"{eventlist_folder}/eventlist_boostedHH4b_{year}.root"

    with uproot.open(file_path) as file:
        for key in file.keys():
            tree = file[key]
            arrays = tree.arrays(library="np")
            df = pd.DataFrame(arrays)

            # Make dict
            dfs_from_root[(year, key)] = df

# test_df = dfs_from_root["2022"] - dfs_from_root["2022EE"]
# print(test_df)
# Display df
for (year, key), df in dfs_from_root.items():
    print(f"DataFrame for year {year}, tree: '{key}':\n{df}\n")